In [37]:
import warnings 
import numpy as np
import pandas as pd
from bokeh.io import show, output_notebook
from bokeh.models import BoxZoomTool, ResetTool, PanTool, HoverTool
from bokeh.models import Plot, Range1d, MultiLine, Circle
from bokeh.models.graphs import from_networkx, NodesAndLinkedEdges, EdgesAndLinkedNodes
from sklearn.neighbors import DistanceMetric
import networkx as nx
import ssl
ssl._create_default_https_context = ssl._create_unverified_context
warnings.filterwarnings('ignore')

In [14]:
df=pd.read_json('https://gist.githubusercontent.com/ozdemirburak/4821a26db048cc0972c1beee48a408de/raw/4754e5f9d09dade2e6c461d7e960e13ef38eaa88/cities_of_turkey.json')
df.columns=['plaka', 'il', 'lat', 'lon', 'nüfus', 'reg.']

In [15]:
df.sample(5)

,plaka,il,lat,lon,nüfus,reg.
49,50,Nevşehir,38.6939,34.6857,286767,İç Anadolu
17,18,Çankırı,40.6013,33.6134,180945,İç Anadolu
29,30,Hakkâri,37.5833,43.7333,278775,Doğu Anadolu
2,3,Afyonkarahisar,38.7507,30.5567,709015,Ege
76,77,Yalova,40.6500,29.2667,233009,Marmara


In [19]:
df=df.reset_index()
df['il']=df['il'].str.lower()
df = df[['il', 'lat', 'lon', 'nüfus']]
df['lat'] = np.radians(df.lat)
df['lon'] = np.radians(df.lon)
dist = DistanceMetric.get_metric('haversine')
dist.pairwise(df[['lat','lon']].to_numpy())*6373 
mesafe=pd.DataFrame(dist.pairwise(df[['lat','lon']].to_numpy())*6373,  columns=df.il.unique(), index=df.il.unique())
mesafe.head(5)

,adana,adıyaman,afyonkarahisar,ağrı,amasya,ankara,antalya,artvin,aydın,balıkesir,...,batman,şırnak,bartın,ardahan,iğdır,yalova,karabük,kilis,osmaniye,düzce
adana,0.000000,5.929604,9.853709,15.904999,7.155555,7.422196,8.962839,14.999917,14.614385,15.327969,...,11.414669,13.943103,10.484355,16.400039,17.764294,13.723897,9.703741,3.536483,1.710463,11.002018
adıyaman,5.929604,0.000000,15.111411,10.009651,7.339310,11.331333,14.799806,9.552115,20.261058,20.509246,...,5.549634,8.206214,13.506344,10.766774,11.855255,18.368597,12.855797,3.028940,4.219237,15.062153
afyonkarahisar,9.853709,15.111411,0.000000,24.325185,10.889875,5.004868,3.635190,22.364889,5.549361,5.475662,...,20.603993,23.312228,6.622452,24.017594,26.198484,4.457109,6.226551,13.339784,11.309824,4.226913
ağrı,15.904999,10.009651,24.325185,0.000000,14.121710,19.796573,24.587354,3.713983,29.743725,29.443631,...,5.152881,4.595543,20.871125,2.784400,1.881649,26.817428,20.451070,12.899771,14.203021,23.191026
amasya,7.155555,7.339310,10.889875,14.121710,0.000000,5.957629,12.353216,11.660041,16.437889,15.559584,...,11.607375,14.363638,6.794990,13.359821,15.927303,12.750822,6.333408,8.030150,6.704902,9.090177


In [22]:
unstacked= mesafe.unstack()
unstacked.index.rename(['şehir A', 'şehir B'], inplace=True)
unstacked = unstacked.to_frame('Aradaki Mesafe km').reset_index()
#unstacked.to_csv(r'edge_list2.txt', header=None, index=None, sep=' ', mode='a')
unstacked.sample(5)

,şehir A,şehir B,Aradaki Mesafe km
2463,hatay,i̇stanbul,16.873115
5301,yozgat,kastamonu,3.648272
104,adıyaman,erzincan,4.524845
2337,gümüşhane,karaman,13.737798
6553,düzce,bartın,2.908794


In [35]:
G = nx.read_weighted_edgelist('/Users/umar/Desktop/edge_list2.txt', delimiter =" ") 
for i in list(G.nodes()): 
    G.nodes[i]['nüfus'] = pd.Series(df['nüfus'].values,index=df['il']).to_dict()[i] 

In [32]:
size=0.5
l=0.5
def box_cox_normalization(node_size):
    from math import ceil
    from math import pow
    compressed_point=(pow(node_size, l) -1)/l
    return ceil(size*compressed_point)

In [42]:
new_sizes=dict(map(lambda node: (node[0] , box_cox_normalization(node[1])), dict(G.degree).items()))
color=dict(map(lambda node: (node[0], 'red' if node[1]>500 else 'blue'), dict(G.degree).items()))
nx.set_node_attributes(G, dict(G.degree), 'connections')
nx.set_node_attributes(G, new_sizes, 'node_size')
plot=Plot(plot_width=700, plot_height=500, x_range=Range1d(-1.1, 1.1), 
          y_range=Range1d(-1.1, 1.1))
plot.add_tools(BoxZoomTool(), ResetTool(), PanTool(), HoverTool(tooltips=[('name', '@name')]))
graph_renderer=from_networkx(G, nx.spring_layout, scale=1, center=(0, 0))

graph_renderer.node_renderer.glyph=Circle(size='node_size')
graph_renderer.inspection_policy = NodesAndLinkedEdges()
graph_renderer.edge_renderer.glyph=MultiLine(line_alpha=0.4, line_width=0.4)

graph_renderer.node_renderer.data_source.data['name'] = df['il'].values
plot.renderers.append(graph_renderer)

output_notebook()
show(plot)

Loading BokehJS ...